# 1. Activate GPU and Install Dependencies

In [1]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


^C


In [58]:
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
torch.cuda.is_available()

True

In [59]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

2935

In [28]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 4            |        cudaMalloc retries: 5         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3386 MiB |   3386 MiB |  23004 MiB |  19617 MiB |
|       from large pool |   3385 MiB |   3385 MiB |  22997 MiB |  19612 MiB |
|       from small pool |      1 MiB |      1 MiB |      6 MiB |      5 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   3386 MiB |   3386 MiB |  23004 MiB |  19617 MiB |
|       from large pool |   3385 MiB |   3385 MiB |  22997 MiB |

# 2. Preprocess data

In [3]:
train_dataset = Dataset.from_pandas(pd.read_csv('../data/movie_reviews/movie_reviews.csv'))
test_dataset = Dataset.from_pandas(pd.read_csv('../data/test_data.csv'))

In [4]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
# tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [6]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [7]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Define the evaluation metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [52]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

In [56]:
from transformers import TrainingArguments, Trainer

num_samples = 40000
batch_size = 8 
num_train_epochs = 3

training_args = TrainingArguments(
    output_dir="finetuning-sentiment-model-40000-movie-reviews-sample",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [60]:
# Train the model
trainer.train()

  0%|          | 0/15000 [00:00<?, ?it/s]

{'loss': 0.2782, 'learning_rate': 1.9337333333333335e-05, 'epoch': 0.1}
{'loss': 0.2646, 'learning_rate': 1.8670666666666667e-05, 'epoch': 0.2}
{'loss': 0.2679, 'learning_rate': 1.8004000000000002e-05, 'epoch': 0.3}
{'loss': 0.252, 'learning_rate': 1.7337333333333334e-05, 'epoch': 0.4}
{'loss': 0.2754, 'learning_rate': 1.6672000000000002e-05, 'epoch': 0.5}
{'loss': 0.2479, 'learning_rate': 1.6005333333333334e-05, 'epoch': 0.6}
{'loss': 0.2671, 'learning_rate': 1.533866666666667e-05, 'epoch': 0.7}
{'loss': 0.2417, 'learning_rate': 1.4672000000000001e-05, 'epoch': 0.8}
{'loss': 0.2477, 'learning_rate': 1.4005333333333335e-05, 'epoch': 0.9}
{'loss': 0.2479, 'learning_rate': 1.3340000000000001e-05, 'epoch': 1.0}
{'loss': 0.1662, 'learning_rate': 1.2673333333333335e-05, 'epoch': 1.1}
{'loss': 0.1564, 'learning_rate': 1.2008000000000003e-05, 'epoch': 1.2}
{'loss': 0.1394, 'learning_rate': 1.1341333333333336e-05, 'epoch': 1.3}
{'loss': 0.1437, 'learning_rate': 1.0674666666666666e-05, 'epoch':

TrainOutput(global_step=15000, training_loss=0.16006983642578124, metrics={'train_runtime': 4787.8547, 'train_samples_per_second': 25.063, 'train_steps_per_second': 3.133, 'train_loss': 0.16006983642578124, 'epoch': 3.0})

# 4. Predict test data

In [61]:
# Upload the model to the Hub
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tiennn/path_to_your_output_dir/commit/3c3d8e31095099ccb4d9be0d11f42c73b93ca09a', commit_message='End of training', commit_description='', oid='3c3d8e31095099ccb4d9be0d11f42c73b93ca09a', pr_url=None, pr_revision=None, pr_num=None)

In [68]:
from transformers import pipeline

sentiment_model = pipeline(model="tiennn/finetuning-sentiment-model-40000-movie-reviews-sample",max_length=512,truncation=True)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

c:\Users\NAME\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NAME\.cache\huggingface\hub\models--tiennn--finetuning-sentiment-model-40000-movie-reviews-sample. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [63]:
def Change(text):
    if text == 'LABEL_0':
        return 0
    return 1

In [64]:
test_df = pd.read_csv('../data/test_data.csv')

In [65]:
test_df['predict'] = [Change(sentiment_model(test_df['text'][i])[0]['label']) for i in test_df.index]

In [67]:
submission_file_df = test_df[['Id','predict']]
submission_file_df.to_csv('submission_file.csv',index=False)